# Automation Cytoscape network creation using RCy3 package
We are going to use the RCy3 package in R to automate the workflow for network creation.
We are using multiple gene epxression datasets acquired from different studies. These datasets contain gene expression profiles of patients with various diseases. 
The diseases which are going to be used in this notebook are: lung-cancer (LC), breast-cancer (BC), metabolically unhealthy obesity (MUO), rheumatoid arthritis (RA), dilated cardiomyopathy (DCM), ishemic cardiomyopathy (ICM) and systemis lupus erythematosus (SLE).

The network will consist of pathways, pathway clusters and the genes which occur in these pathways. These pathways are acquired based on genes associated with inflammation which were retrieved from [DisGeNET](http://www.disgenet.org/home/) and [GeneCards](https://www.genecards.org/). 

In [1]:
# check wd
getwd()

[1] "C:/Users/Laurent/Documents/test"

In [2]:
# load library
library(RCy3)

### We first start with opening cytoscape and checking if we are connected

In [14]:
# check if cytoscape is open and check version
cytoscapePing()
cytoscapeVersionInfo()

[1] "You are connected to Cytoscape!"

apiVersion cytoscapeVersion 
            "v1"          "3.7.1"

### Network creation
To begin with, we will load in our node and edge files to create our network.

We have to clean these files so they are usable for network creation.

In [15]:
# load in data of network
nodes <- as.data.frame(read.table(file.path(getwd(), "Network", "nodes_final.txt"), header = T, sep = "\t", stringsAsFactors = FALSE))
edges <- as.data.frame(read.table(file.path(getwd(), "Network", "edges_final.txt"), header = T, sep = "\t", stringsAsFactors = FALSE))

# clean up data
colnames(nodes)[1] <- "id"
nodes$id <- as.character(nodes$id)
edges <- edges[,c(-3,-4)]
colnames(edges)[c(1,2)] <- c("source", "target") 
edges$interaction <- "interacts"
edges$target <- as.character(edges$target)

head(nodes)
head(edges)

id,type,entrezgene
AKT3,Gene,10000
CDH3,Gene,1001
SOD2,Gene,100129518
MIR718,Gene,100313781
FAM13A,Gene,10144
MIR6886,Gene,102465534


source,target,interaction
Vitamin_B12,PPARA,interacts
Vitamin_B12,C4A,interacts
Vitamin_B12,TF,interacts
Vitamin_B12,LCN2,interacts
Vitamin_B12,ELANE,interacts
Vitamin_B12,BPI,interacts


Now we can create our network in Cytoscape

In [16]:
# create network from the data frames and map column for entrezgene IDs
createNetworkFromDataFrames(nodes, edges, title = "MyNetwork", collection = "MyCollection")

Loading data...
Applying default style...
Applying preferred layout...


networkSUID 
         52

We have to load in our gene expression data file, and merge this file with the table in cytoscape.

In [17]:
# load data set with gene expression values (logFC, p.value)
expr_data <- read.table(file.path(getwd(), "expr_data", "merged_data_final.txt"), header = T, sep ="\t")

# load data into cytoscape
loadTableData(expr_data, data.key.colum = "entrezgene", table.key.column = "entrezgene")

# check if tables are well merged
nodeTable <- getTableColumns(table = "node")
head(nodeTable)

[1] "Success: Data loaded in defaultnode table"

,SUID,shared name,name,selected,id,entrezgene,type,hgnc_symbol,logFC_BC,PValue_BC,...,logFC_MUO,PValue_MUO,logFC_RA,PValue_RA,logFC_DCM,PValue_DCM,logFC_ICM,PValue_ICM,logFC_SLE,PValue_SLE
62,62,AKT3,AKT3,FALSE,AKT3,10000,Gene,AKT3,-3.6182060,0.00000446,...,0.158046200,0.03617197,-0.091532500,0.5187737000,-0.1170548,0.53705530,-0.00574414,0.9794696,0.06630566,0.2096076
63,63,CDH3,CDH3,FALSE,CDH3,1001,Gene,CDH3,0.5611705,0.71538600,...,0.009146553,0.72093250,0.005623439,0.9375439000,1.3844880,0.04941398,0.90182150,0.2080566,0.08489361,0.7994946
64,64,SOD2,SOD2,FALSE,SOD2,100129518,Gene,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
65,65,MIR718,MIR718,FALSE,MIR718,100313781,Gene,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
66,66,FAM13A,FAM13A,FALSE,FAM13A,10144,Gene,FAM13A,-3.8751590,0.00006440,...,-0.477643700,0.00004010,-1.094291000,0.0004184986,0.1338444,0.54133960,0.12945470,0.5958564,0.00000000,1.0000000
67,67,MIR6886,MIR6886,FALSE,MIR6886,102465534,Gene,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


### Manual visualization 
Before we start to visualize our network, we have to perform some manual stuff! 

To analyze the network we have to go to Cytoscape. First Open cytoscape go to Tools then NetworkAnalyzer then Network Analysis then Analyze Network. Treat the network as unidirected and click OK. We analyzed the network now we want to change the layout of the network. Go to Layout then yFiles Organic Layout. Close this message box when finished, and return to R.

### Automated visualization
Now everthing is loaded into cytoscape and the manual stuff is done, we can working on the automatization of the visual style.

For every dataset we would like to create a own network. We will do this using the clone network function.

When we have a network available for every dataset we have, we can set a visual style which maps the gene expression (logFC) and significance (p-value) for every gene node in the network per dataset. Fill in for the Automation function y, z and p. Whereas; y is the negative logFC cutoff, z is the positive logFC cutoff and p is the p-value.

In [18]:
# Fill in for the Automation function y, z and p. 
# Whereas negLogFC is the negative logFC cutoff, posLogFC is the positive logFC cutoff and pValue is the p-value.
pValue <- 0.05
negLogFC <- -1.00
posLogFC <- 1.00

In [20]:
Automation <- function(y,z,p) {
    
# lock width and height of nodes
lockNodeDimensions(TRUE)
    
# we have 7 gene expression sets so we repeat this function 8 times, we will use a repeat loop for this
diseases <- c("BC","LC","MUO","RA","DCM","ICM","SLE")

x <- 1
repeat {
  cloneNetwork(network = "MyNetwork")
  setCurrentNetwork(network="MyNetwork_1")
  renameNetwork(diseases[x])
  x = x + 1
  if (x == length(diseases)+1){
  break}
}
    
# Map logFC and pValue per disease for every respective network and add visual style to every network
x <- 1
repeat {
  setCurrentNetwork(network = diseases[x])
  
  style.name = paste("style_", diseases[x], sep = "")
  
  # map logFC of gene expression data per disease on respective networks
  mappings <- list(nodeFill <- mapVisualProperty("node fill color", table.column = paste("logFC_", diseases[x] , sep = ""), 
                                    mapping.type = "continuous",
                                    c(y,0,z),
                                    c("#0000FF", "#FFFFFF", "#FF0000")),
  
  # map significance per disease on respective networks
  nodeBorder <- mapVisualProperty("node border paint", table.column = paste("PValue_", diseases[x], sep = ""),
                                    mapping.type = "continuous",
                                    table.column.values = c(0.00, p, 1.00), 
                                    c("#00FF00", "#FFFFFF", "#FFFFFF")),
  
  # map node shapes
  nodeShape <- mapVisualProperty("Node Shape", table.column = "type", 
                                    mapping.type = "discrete", 
                                    table.column.values = c("Gene", "Process", "InflGene"),
                                    c("DIAMOND", "ELLIPSE", "VEE")),
  
  label <- mapVisualProperty("Node Label", table.column = "shared name",
                             mapping.type = "passthrough"))

  defaults <- list(NODE_FILL_COLOR = "#999999",
                   NODE_BORDER_PAINT = "#999999",
                   NODE_BORDER_WIDTH = 7,
                   NODE_LABEL_FONT_SIZE = 18,
                   NETWORK_TITLE = diseases[x])
  
  # create and set unique visual style per disease dataset
  createVisualStyle(style.name, defaults, mappings = mappings)
  setVisualStyle(style.name)

  x = x + 1
  if (x == length(diseases)+1){
    break}
}

# save disease networks
x <- 1
repeat {
  setCurrentNetwork(network = diseases[x])
  fitContent()
  
  png.file <- file.path(getwd(), "Images", "networks", paste0("logFC", z), paste0("gene_expr_network", diseases[x], ".png"))
  exportImage(png.file, type = "png", resolution=600, zoom=500)
  
  x = x + 1
  if (x == length(diseases)+1){
    break}
}

# create subnetwork of significant differentially expressed genes
x <- 1
repeat {
  setCurrentNetwork(network = diseases[x])
  
  createColumnFilter(filter.name = "expr genes",     column = paste("logFC_", diseases[x], sep = "") , c(y,z), "IS_NOT_BETWEEN")
  createColumnFilter(filter.name = "sig genes",      column = paste("PValue_", diseases[x], sep = ""), p, "LESS_THAN")
  sigexpr <- createCompositeFilter('combined filter', filter.list = c("sig genes", "expr genes"), type = "ALL")
  
  process <- createColumnFilter(filter.name = "Pathway filter", column = "type", "Process", "IS")
  
  selectNodes(nodes = c(process$nodes, sigexpr$nodes), by.col = "shared name")
  
  createSubnetwork(nodes = "selected", subnetwork.name = paste0("subnetwork ", diseases[x], " genes"))
  
  x = x + 1
  if (x == length(diseases)+1){
    break}
}

# create subnetwork of significant differentially expressed inflammation genes
x <- 1
repeat {
  setCurrentNetwork(network = diseases[x])
  
  createColumnFilter(filter.name = "Infl filter",    column = "type", "InflGene", "IS")
  createColumnFilter(filter.name = "expr genes",     column = paste("logFC_", diseases[x], sep = "") , c(y,z), "IS_NOT_BETWEEN")
  createColumnFilter(filter.name = "sig genes",      column = paste("PValue_", diseases[x], sep = ""), p, "LESS_THAN")
  sigexpr <- createCompositeFilter('combined filter', filter.list = c("Infl filter", "sig genes", "expr genes"), type = "ALL")
  
    process <- createColumnFilter(filter.name = "Pathway filter", column = "type", "Process", "IS")
  
  selectNodes(nodes = c(process$nodes, sigexpr$nodes), by.col = "shared name")
  
  createSubnetwork(nodes = "selected", subnetwork.name = paste0("subnetwork ", diseases[x], " inflGenes"))
  
  x = x + 1
  if (x == length(diseases)+1){
    break}
}

# save subnetworks genes
subgenes <- c("subnetwork BC genes","subnetwork LC genes","subnetwork MUO genes",
              "subnetwork RA genes","subnetwork DCM genes",
              "subnetwork ICM genes","subnetwork SLE genes")
x <- 1
repeat {
  setCurrentNetwork(network = subgenes[x])
  fitContent()
  
  png.file <- file.path(getwd(), "Images", "subGeneNetworks", paste0("logFC", z), paste0("subgene_expr_network", diseases[x], ".png"))
  exportImage(png.file, type = "png", resolution=600, zoom=500)
  
  x = x + 1
  if (x == length(diseases)+1){
    break}
}

# save subnetworks inflammation genes
subInflGenes <- c("subnetwork BC inflGenes","subnetwork LC inflGenes","subnetwork MUO inflGenes",
              "subnetwork RA inflGenes","subnetwork DCM inflGenes",
              "subnetwork ICM inflGenes","subnetwork SLE inflGenes")
x <- 1
repeat {
  setCurrentNetwork(network = subInflGenes[x])
  fitContent()
  
  png.file <- file.path(getwd(), "Images", "subInflGeneNetworks", paste0("logFC", z), paste0("subinflgene_expr_network", diseases[x], ".png"))
  exportImage(png.file, type = "png", resolution=600, zoom=500)
  
  x = x + 1
  if (x == length(diseases)+1){
    print("Done!")
    break}
}

# save Cytoscape session file
session.file <- file.path(getwd(), "Network", paste0(z, "gene_expr_networks.cys"))
saveSession(session.file)
}

# values were filled in above
Automation(y = negLogFC, z = posLogFC, p = pValue)

[1] "Done!"


In [9]:
# information about session
sessionInfo()

R version 3.5.1 (2018-07-02)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 10 x64 (build 17134)

Matrix products: default

locale:
[1] LC_COLLATE=Dutch_Netherlands.1252  LC_CTYPE=Dutch_Netherlands.1252   
[3] LC_MONETARY=Dutch_Netherlands.1252 LC_NUMERIC=C                      
[5] LC_TIME=Dutch_Netherlands.1252    

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] RCy3_2.2.6           RevoUtils_11.0.1     RevoUtilsMath_11.0.0

loaded via a namespace (and not attached):
 [1] igraph_1.2.2        graph_1.60.0        Rcpp_1.0.0         
 [4] magrittr_1.5        BiocGenerics_0.28.0 uuid_0.1-2         
 [7] R6_2.3.0            httr_1.4.0          tools_3.5.1        
[10] parallel_3.5.1      R.oo_1.22.0         htmltools_0.3.6    
[13] digest_0.6.18       crayon_1.3.4        RJSONIO_1.3-1.1    
[16] IRdisplay_0.7.0     repr_0.19.1         base64enc_0.1-3    
[19] R.utils_2.7.0       curl_3.3    